In [2]:
import torch
import torchvision
from torchvision import transforms
from MLProject2Dataset import *

In [2]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [10]:
dataloader = MLProject2Dataset('./data/dermoscopy_classification')
generator = torch.Generator().manual_seed(42)
train, validate, test = torch.utils.data.random_split(dataloader, [0.6, 0.1, 0.3], generator)

In [3]:
len(train), len(validate), len(test)

(6010, 1001, 3004)

In [4]:
image_transforms(10, 2)

Compose(
    Resize(size=(10, 2), interpolation=bilinear, max_size=None, antialias=warn)
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [8]:
# data_dir = './data/dermoscopy_classification/'
# metadata_fname = 'metadata.csv'


# files = pd.DataFrame({
#             'image_id': [Path(path).stem for path in glob.glob(f'{data_dir}/*/*')],
#             'path': [os.path.abspath(path) for path in glob.glob(f'{data_dir}/*/*')]
#         })

# metadata = pd.read_csv(data_dir + '/' + metadata_fname)
# metadata['dx'] = metadata['dx'].astype('category').cat.codes

# dataset = pd.merge(files, metadata[['dx', 'image_id']], on='image_id')


In [ ]:
# import torch
# from torch import nn
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from torch import optim
# import torchvision
# import pandas as pd
# import glob
# import os
# from pathlib import Path
# import PIL

In [ ]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.params(), lr=0.0001)